# Firefly Encoding

This project demonstrates Firefly Encoding. 

## Using this Notebook

The easiest way to use this notebook is simply "Cell > Run All" or "Kernel > Restart & Run All"

## Firefly Code

Firefly is a visual [Substitution Cipher](https://en.wikipedia.org/wiki/Substitution_cipher), designed using a quaternary encoding scheme. It accepts keyboard characters (referred to as "symbols") as input, and produces color sequences (called "patterns") as output.

Encodable symbols are limited to a subset of the keys and characters which can be typed by a standard US keyboard.

Each encoded pattern is 4 "units" long. Each unit has 4 possible states corosponding to the primary colors Red, Green, Blue, and None (or "off").

Each symbol we wish to encode is explicitly defined along with its pattern (the "encoded" or "enciphered" value). For example the symbol "a" is encoded to become the pattern "Red, Red, Red, Red." It follows that conversely, the pattern "Red, Red, Red, Red" is correctly decoded as the symbol "a".

### Encoding

This software renders a simple text editor application which appears as a standalone window. It does this using the Qt framework. All keystrokes sent to the application are intercepted and encoded as RGB light patterns which are then transmitted to a peripheral device capable of displaying color sequences. This software is designed to display encoded patterns on a [Razer Firefly v1](https://www.razer.com/gaming-mouse-mats/) mouse mat (thus the name). The Firefly is a purely decorative device that is easily controlled via USB.

The Firefly features 15 RGB LEDs, five on each of three sides. The "back" (or "top") has no lights. A sequence of 15 colors displayed on the Firefly is called a "frame." Each frame is held for a fixed time period, and then a new frame is generated. This cycle repeats as long as the software is running. When no encoded values are present, the software is designed to produce "noise" in the form of an color-chase sequence. The sequence Red, Yellow, Green, Cyan, Blue, Magenta is repeated continuously. For each subsequent noise frame, the existing pattern is rotated by one color.

Encoded patterns are simply inserted into the color-chase sequence. Each pattern to be encoded is inserted only once. Multiple repetitions of the same pattern indicate multiple symbols have been encoded. Designed in this way, encoded patterns appear to "walk" through the existing color-chase sequence.

### Decoding

A four unit code was chosen so that a would-be decoder only needs to "see" one side of the mat. One LED position is intended to serve as an "alignment" light, and the other four visible LEDs are used to encode data. The color white (which for an RGB LED means "all colors on") is used for alignment. Because the encoded patterns move quickly, a video recording and/or programmatic analysis are preferred, but decoding can be accomplished manually.

Decoding is simply a matter of reading the color pattern which appears on the Firefly with respect to the alignment light and then looking up the corrosponding symbol in the decode table.

For example, if a captured frame of the Firefly shows: "White, Red, Red, Red, Red," then we know we have a valid frame because the color white appears in the first position. The encoded pattern is then determined to be "Red, Red, Red, Red," which maps to the symbol "a".

In [1]:
# Python imports
from collections import deque
from itertools import chain, islice

import usb.backend.libusb0
import usb.core

from PyQt5.QtCore import *
from PyQt5.QtGui import *
from PyQt5.QtWidgets import *

### Configurables

There are only a few options:
`n_blank_after` and `n_blank_before` indicate the number of LEDs which should preceed and follow each encoded character. At least one of these should be non-zero in order to help the decoding party separate encoded patterns from the surrounding visual noise.

`frame_interval` indicates the time (in ms) to hold each frame.

In [2]:
firefly_enabled = True
frame_interval = 200
scroll_mode = False

n_blank_after = 0
n_blank_before = 0

### Code Key

Two Python dictionaries are defined here. Nonprintable and printable characters are defined separately as an artifact of the implementation. The keys of each dictionary represent the encodable symbols and the corrosponding values represented encoded patterns. Each pattern is a four-element list comprised of RGB color values. In this implementaiton, those color values are specified as standard 3 byte RGB values (each represented as a 3 item list-of-integers).

This code is designed for ease of comprehension and as an illustration. It is not optimized at all.

In [3]:
X = [0,0,0]
R = [255,0,0]
G = [0,255,0]
B = [0,0,255]
C = [0,255,255]
M = [225,0,255]
Y = [255,225,0]
W = [255,255,255]

key_symbols = {
    Qt.Key_Backspace: [G,X,R,R],
    Qt.Key_Delete:    [G,X,R,G],
    Qt.Key_Enter:     [G,X,R,B],
    Qt.Key_Return:    [G,X,R,X],
    Qt.Key_Space:     [G,X,G,R],
    Qt.Key_Home:      [G,X,G,G],
    Qt.Key_End:       [G,X,G,B],
    Qt.Key_Left:      [G,X,G,X],
    Qt.Key_Up:        [G,X,B,R],
    Qt.Key_Right:     [G,X,B,G],
    Qt.Key_Down:      [G,X,B,B],
    Qt.Key_PageUp:    [G,X,B,X],
    Qt.Key_PageDown:  [G,X,X,R],
    Qt.Key_Escape:    [G,X,X,G],
    Qt.Key_Tab:       [G,X,X,B],
#   None:             [G,X,X,X],
    }

text_symbols = {
    '!': [G,R,R,R],# G,G,G,G
    '"': [G,R,R,G],# G,G,B,B
    '#': [G,R,R,B],# G,G,B,R
    '$': [G,R,R,X],# G,G,B,G
    '%': [G,R,G,R],# G,G,R,B
    '&': [G,R,G,G],# G,G,R,R
    "'": [G,R,G,B],# G,G,R,G
    '(': [G,R,G,X],# G,G,G,B
    ')': [G,R,B,R],# G,G,G,R
    '*': [G,R,B,B],# G,B,B,G
    '+': [G,R,B,G],# G,B,B,B
    ',': [G,R,B,X],# G,B,B,R
    '-': [G,R,X,R],# G,B,B,G
    '.': [G,R,X,G],# G,B,B,B
    '/': [G,R,X,B],# G,B,B,R
    ':': [G,R,X,X],# G,R,B,B
    ';': [G,G,R,R],# G,R,B,R
    '<': [G,G,R,G],# G,R,B,G
    '=': [G,G,R,B],# G,R,R,B
    '>': [G,G,R,X],# G,R,R,R
    '?': [G,G,G,R],# G,R,G,G
    '@': [G,G,G,G],# G,R,G,B
    '[': [G,G,G,B],# G,G,R,B
    '\\':[G,G,G,X],# G,G,R,R
    ']': [G,G,B,R],# G,G,B,G
    '^': [G,G,B,G],# G,G,B,B
    '_': [G,G,B,B],# G,G,B,R
    '`': [G,G,B,X],# G,G,G,B
    '{': [G,G,X,R],# G,R,B,R
    '|': [G,G,X,G],# G,R,R,R
    '}': [G,G,X,B],# G,R,G,R
    '~': [G,G,X,X],# G,R,R,B

    '0': [G,B,R,R],# R,B,X,X
    '1': [G,B,R,G],# R,B,B,X
    '2': [G,B,R,B],# R,B,B,B
    '3': [G,B,R,X],# R,G,X,X
    '4': [G,B,G,R],# R,G,G,X
    '5': [G,B,G,G],# R,G,G,G
    '6': [G,B,G,B],# R,R,X,X
    '7': [G,B,G,X],# R,R,R,X
    '8': [G,B,B,R],# R,R,R,R
    '9': [G,B,B,G],# R,B,G,X

    'A': [B,R,R,R],# B,G,G,B
    'B': [B,R,R,G],# B,G,G,R
    'C': [B,R,R,B],# B,G,B,G
    'D': [B,R,G,R],# B,G,B,B
    'E': [B,R,G,G],# B,G,B,R
    'F': [B,R,G,B],# B,G,R,B
    'G': [B,R,B,R],# B,G,R,R
    'H': [B,R,B,G],# B,G,R,G
    'I': [B,R,B,B],# B,B,G,B
    'J': [B,G,R,R],# B,B,G,R
    'K': [B,G,R,G],# B,B,G,G
    'L': [B,G,R,B],# B,B,B,B
    'M': [B,G,G,R],# B,B,B,R
    'N': [B,G,G,G],# B,B,B,G
    'O': [B,G,G,B],# B,B,R,B
    'P': [B,G,B,R],# B,B,R,R
    'Q': [B,G,B,G],# B,B,R,G
    'R': [B,G,B,B],# B,R,G,B
    'S': [B,B,R,R],# B,R,G,R
    'T': [B,B,R,G],# B,R,G,G
    'U': [B,B,R,B],# B,R,B,B
    'V': [B,B,G,R],# B,R,B,R
    'W': [B,B,G,G],# B,R,B,G 
    'X': [B,B,G,B],# B,R,R,B
    'Y': [B,B,B,R],# B,R,R,R
    'Z': [B,B,B,G],# B,R,R,G

    'a': [R,R,R,R],# R,G,G,B
    'b': [R,R,R,G],# R,G,G,R
    'c': [R,R,R,B],# R,G,B,G
    'd': [R,R,G,R],# R,G,B,B
    'e': [R,R,G,G],# R,G,B,R
    'f': [R,R,G,B],# R,G,R,B
    'g': [R,R,B,R],# R,G,R,R
    'h': [R,R,B,G],# R,G,R,G
    'i': [R,R,B,B],# R,B,G,B
    'j': [R,G,R,R],# R,B,G,R
    'k': [R,G,R,G],# R,B,G,G
    'l': [R,G,R,B],# R,B,B,B
    'm': [R,G,G,R],# R,B,B,R
    'n': [R,G,G,G],# R,B,B,G
    'o': [R,G,G,B],# R,B,R,B
    'p': [R,G,B,R],# R,B,R,R
    'q': [R,G,B,G],# R,B,R,G
    'r': [R,G,B,B],# R,R,G,B
    's': [R,B,R,R],# R,R,G,R
    't': [R,B,R,G],# R,R,G,G
    'u': [R,B,R,B],# R,R,B,B
    'v': [R,B,G,R],# R,R,B,R
    'w': [R,B,G,G],# R,R,B,G
    'x': [R,B,G,B],# R,R,R,B
    'y': [R,B,B,R],# R,R,R,R
    'z': [R,B,B,G],# R,R,R,G
    }

### Peripheral Driver

The following code is designed to drive a Razer Firefly USB mousepad. The [openrazer](https://github.com/openrazer/openrazer/) project was instrumental in defining these classes.

In [4]:
class RazerChromaCommand:
    def __init__(self, *args, transaction_id=0xff, command_class=0xff, command_id=0x00):
        self.report = [
            0x00,          # status - new command
            transaction_id,
            0x00,          # remaining_packets[0]
            0x00,          # remaining_packets[1]
            0x00,          # protocol_type
            len(args),     # data_size
            command_class, # command_class
            command_id     # command_id
        ]
        
        self.report += args
        self.report += [0x00] * (80 - len(args))
        
        crc = 0
        for i in range(2, 88):
            crc ^= self.report[i]
        self.report += [crc]  # crc - xor'ed bytes of report
        self.report += [0x00] # reserved
        
        assert len(self.report) == 90


class RazerDevice:
    idVendor = 0x1532

    def __init__(self):
        backend = usb.backend.libusb0.get_backend()
        devices = list(
            usb.core.find(idVendor=self.idVendor,
                          idProduct=self.idProduct,
                          find_all=True,
                          backend=backend)
        )
        
        if len(devices) < 1:
            raise UserWarning('No Devices Found')
            
        self.usb_device = devices[0]
        self.configuration = self.usb_device[0]
        self.interface = self.configuration[(0,0)]
        assert self.interface.bInterfaceProtocol == 0x2
        self.usb_device.set_configuration()
    
    def send_control_message(self, command):
        sent = self.usb_device.ctrl_transfer(
            0x21,      # request_type - USB_TYPE_CLASS | USB_RECIP_INTERFACE | USB_DIR_OUT
            0x09,      # request - HID_REQ_SET_REPORT
            0x300,     # value
            0x00,      # report_index
            command.report
        )
        if sent != 90:
            raise UserWarning('USB ctrl_transfer failed')
        response = self.usb_device.ctrl_transfer(
            0xA1,  # request_type,
            0x01,  # request
            0x300, # value
            0x00,  # response_index
            90
        )
        # TODO - check the response against our last message
        # packets_remaining, command_id, command_class and transaction_id should be equal
        # status:
        # 1 = Busy
        # 2 = Success
        # 3 = Failure
        # 4 = Timeout
        # 5 = Not Supported
        if response[0] != 2:
            raise UserWarning('USB error')
        return response


class RazerFirefly(RazerDevice):
    idProduct = 0x0c00

    def set_custom_frame(self, rgb_data):
        if len(rgb_data) != 15:
            raise UserWarning('rgb_data must be [r,g,b]*n_leds')

        data = [
            0x00,     # start_col
            0x0e,     # stop_col
            #int(n/3)-1
        ]
        data += sum(rgb_data, [])
        
        return self.send_control_message(
            RazerChromaCommand(
                *data,
                transaction_id = 0x3e,
                command_class = 0x03,
                command_id = 0x0c,
            ))

    def set_effect_custom(self):
        return self.send_control_message(
            RazerChromaCommand(
                0x05, 0x00,
                transaction_id = 0x3d,
                command_class = 0x03,
                command_id = 0x0a,
            ))
    
    def set_effect_wave(self, ccw=False):
        return self.send_control_message(
            RazerChromaCommand(
                0x01, 0x01 if ccw else 0x02,
                transaction_id = 0x3e,
                command_class = 0x03,
                command_id = 0x0a,
            ))

### Encoder

The following code acts as a bridge between the UI and the Firefly peripheral, and it is where the encoding happens.

To bridge the UI, a custom `QPlainTextEdit` widget has been defined. This widget instantiates the Encoder class, which in turn attaches to the Firefly peripheral. `QThreadPool` and `QRunnable` classes could be [added to improve performance](https://www.learnpyqt.com/tutorials/multithreading-pyqt-applications-qthreadpool/), but simple testing has shown performance to be adequate.

In [5]:
class Encoder:
    def __init__(self):
        self.firefly = RazerFirefly()
        self.rgb_data = deque([])
        self.rgb_pattern = deque([R, Y, G, C, B, M])
        self.scroll_mode = scroll_mode

    def fill_rgb_data(self, size):
        while(len(self.rgb_data) < size):
            self.rgb_data.append(self.rgb_pattern[0])
            self.rgb_pattern.rotate(-1)
        
    def handle_keyevent(self, key, text):
        """
        Handle keyboard input
            key - the numeric QtCore.Qt.Key_? value of the KeyEvent
            text - unicode representation of the text in the KeyEvent
        """
        if len(text) != 1:
            return
        self.rgb_data.extend([X]*n_blank_before)
        self.rgb_data.extend([W]) # alignment light
        if text in text_symbols:
            self.rgb_data.extend(text_symbols[text])
        elif key in key_symbols:
            self.rgb_data.extend(key_symbols[key])
        else:
            self.rgb_data.extend([W,W,W,W])
        self.rgb_data.extend([X]*n_blank_after)
        
    def next_frame(self):
        """
        Display the next frame of output.
        
        In scroll mode:
        If there is encoded data in the `rgb_data` buffer, simply advance
        the light sequence by one position. Fill in any blank data with an 
        RGB chase pattern. This will be called regularly.
        
        In non-scroll mode:
        Only the front/middle 5 LEDs will be lit.
        """

        if self.scroll_mode:
            self.fill_rgb_data(15)
            frame = list(islice(self.rgb_data, 0, 15))
            self.rgb_data.popleft()
        else:
            self.fill_rgb_data(5)
            frame = [X,X,X,X,X]
            frame.extend(list(islice(self.rgb_data, 0, 5)))
            frame.extend([X,X,X,X,X])
            for i in range(5):
                self.rgb_data.popleft()
        self.firefly.set_custom_frame(frame)
        self.firefly.set_effect_custom()


class PlainTextEdit(QPlainTextEdit):
    def __init__(self, *args):
        super().__init__(*args)
        
        self.zoomIn(10)
        
        self.encoder = Encoder()
        self.timer = QTimer()
        self.timer.setInterval(frame_interval)
        self.timer.timeout.connect(self.encoder.next_frame)
        self.timer.start()

    def keyPressEvent(self, e):
        if firefly_enabled:
            self.encoder.handle_keyevent(e.key(), e.text())
        super().keyPressEvent(e)

### UI Code

This UI is a simple text editor application taken from the PyQt [code examples](https://github.com/pyqt/examples/blob/_/src/07%20Qt%20Text%20Editor/main.py).

Only one line has been changed; instead of the standard `QPlainTextEdit` widget, the modified widget defined above has been substituted.

In [6]:
class MainWindow(QMainWindow):
    def closeEvent(self, e):
        if not text.document().isModified():
            return
        answer = QMessageBox.question(
            window, None,
            "You have unsaved changes. Save before closing?",
            QMessageBox.Save | QMessageBox.Discard | QMessageBox.Cancel
        )
        if answer & QMessageBox.Save:
            save()
        elif answer & QMessageBox.Cancel:
            e.ignore()

app = QApplication([])
app.setApplicationName("Text Editor")
# Use our custom text editor widget
text = PlainTextEdit()
window = MainWindow()
window.setCentralWidget(text)

file_path = None

menu = window.menuBar().addMenu("&File")
open_action = QAction("&Open")
def open_file():
    global file_path
    path = QFileDialog.getOpenFileName(window, "Open")[0]
    if path:
        text.setPlainText(open(path).read())
        file_path = path
open_action.triggered.connect(open_file)
open_action.setShortcut(QKeySequence.Open)
menu.addAction(open_action)

save_action = QAction("&Save")
def save():
    if file_path is None:
        save_as()
    else:
        with open(file_path, "w") as f:
            f.write(text.toPlainText())
        text.document().setModified(False)
save_action.triggered.connect(save)
save_action.setShortcut(QKeySequence.Save)
menu.addAction(save_action)

save_as_action = QAction("Save &As...")
def save_as():
    global file_path
    path = QFileDialog.getSaveFileName(window, "Save As")[0]
    if path:
        file_path = path
        save()
save_as_action.triggered.connect(save_as)
menu.addAction(save_as_action)

close = QAction("&Close")
close.triggered.connect(window.close)
menu.addAction(close)

help_menu = window.menuBar().addMenu("&Help")
about_action = QAction("&About")
help_menu.addAction(about_action)
def show_about_dialog():
    text = "<center>" \
           "<h1>Text Editor</h1>" \
           "&#8291;" \
           "<img src=icon.svg>" \
           "</center>" \
           "<p>Version 31.4.159.265358<br/>" \
           "Copyright &copy; Company Inc.</p>"
    QMessageBox.about(window, "About Text Editor", text)
about_action.triggered.connect(show_about_dialog)

### Invocation

Display the Qt window and run the application

In [7]:
window.show()
app.exec_()

0

### Exit

When the application exits, set the Firefly to its wave effect to be friendly.

In [8]:
text.encoder.firefly.set_effect_wave()

array('B', [2, 62, 0, 0, 0, 2, 3, 10, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 8, 0])